## Neural Network Example

<img src="https://camo.githubusercontent.com/269f47b8185a2ca349ead57db511250553fd918b/687474703a2f2f63733233316e2e6769746875622e696f2f6173736574732f6e6e312f6e657572616c5f6e6574322e6a706567" />

### TLDR;

1. 数据一定要进行规一化处理，防止出现 NaN.


In [117]:
"""Neural Network

A 2-Hidden Layers Fully Connected Neural Network (a.k.a Multilayer Perceptron)
implementation with TensorFlow. This example is using the MNIST database
of handwritten digits (http://yann.lecun.com/exdb/mnist/).

"""

import numpy as np
import tensorflow.compat.v1 as tf

tf.disable_eager_execution()

In [118]:
def load_dataset(epochs=None, batch_size=None):
    """
    load mnist dataset, train((60000, 28, 28), ()), test((10000, 28, 28), ())
    """
    mnist = tf.keras.datasets.mnist
    train, test = mnist.load_data()
    train_dataset = tf.data.Dataset.from_tensor_slices(train)
    test_dataset = tf.data.Dataset.from_tensor_slices(test)
    train_dataset = train_dataset.map(lambda x, y: (tf.reshape(x / 255, [784,]), tf.one_hot(y, 10)))
    test_dataset = test_dataset.map(lambda x, y:(tf.reshape(x / 255, [784,]), tf.one_hot(y, 10)))
    train_dataset = train_dataset.shuffle(60000)
    test_dataset = test_dataset.shuffle(10000)
    
    if epochs is not None:
        train_dataset = train_dataset.repeat(epochs)
    if batch_size is not None:
        train_dataset = train_dataset.batch(batch_size)
        test_dataset = test_dataset.batch(batch_size)
    
    return train_dataset, test_dataset

In [119]:
def glorot(n1, n2):
    """
    Glorot Xiavier Initialization
    """
    return np.sqrt(2.0 / (n1 + n2))

def NN_Model():
    n_hidden_1 = 32
    n_hidden_2 = 32
    num_input = 784
    num_classes = 10
    
    glorot = tf.glorot_normal_initializer()
    
    X = tf.placeholder(tf.float32, [None, num_input])
    Y = tf.placeholder(tf.float32, [None, num_classes])
    
    weights = {
        'h1': tf.Variable(glorot([num_input, n_hidden_1])),
        'h2': tf.Variable(glorot([n_hidden_1, n_hidden_2])),
        'out': tf.Variable(glorot([n_hidden_2, num_classes])),
    }
    biases = {
        'h1': tf.Variable(tf.zeros([n_hidden_1,])),
        'h2': tf.Variable(tf.zeros([n_hidden_2,])),
        'out': tf.Variable(tf.zeros([num_classes,])),
    }
    
#     z2 = tf.matmul(weights['h1'], X)
    z2 = tf.add(tf.matmul(X, weights['h1']), biases['h1'])
    a2 = tf.nn.leaky_relu(z2)
    z3 = tf.add(tf.matmul(a2, weights['h2']), biases['h2'])
    a3 = tf.nn.leaky_relu(z3)
    z4 = tf.add(tf.matmul(a3, weights['out']), biases['out'])
    y_pred = tf.nn.softmax(z4)
#     y_pred = z4
    
    cross_entropy = -Y * tf.log(y_pred)
    loss = tf.reduce_sum(tf.reduce_sum(cross_entropy, reduction_indices=[1]))
    
    return X, Y, y_pred, loss

In [126]:
n_samples = 60000
learning_rate = 0.1
epochs = 10
batch_size = 100

def train():
    X, Y, y_pred, loss = NN_Model()
    optimizer = tf.train.AdamOptimizer().minimize(loss)
    
    train_dataset, test_dataset = load_dataset(epochs, batch_size)
    train_next = tf.data.make_one_shot_iterator(train_dataset).get_next()
    test_next = tf.data.make_one_shot_iterator(test_dataset).get_next()
    
    init = tf.global_variables_initializer()
    
    with tf.Session() as sess:
        sess.run(init)
        
        # Train the model
        for i in range(epochs):
            total_loss = 0.0
            n_batches = int(n_samples / batch_size)
            for j in range(n_batches):
#                 if j > 2:
#                     break
                X_train, Y_train = sess.run(train_next)
                feed_dict = {X: X_train, Y: Y_train}
                _, loss_val, y_hat = sess.run([optimizer, loss, y_pred], feed_dict)
                total_loss += loss_val
            print('Epoch %d: loss = %5.3f' % (i+1, total_loss / n_samples))
        print('Train Finished!')
        
        # Test the model
        acc = 0
        n_test = 0
        try:
            X_test, Y = sess.run(test_next)
            Y_hat = sess.run(y_pred, feed_dict={X: X_test})
            Y_hat_idx = np.argmax(Y_hat, axis=1)
            Y_idx = np.argmax(Y, axis=1)
            acc += np.sum(Y_hat_idx == Y_idx)
            n_test += len(Y_idx)
        except tf.errors.OutOfRangeError:
            print('Test Finished!')
        
        print('Accuracy: %5.3f' % (acc / n_test))               

In [127]:
train()

Epoch 1: loss = 0.450
Epoch 2: loss = 0.222
Epoch 3: loss = 0.180
Epoch 4: loss = 0.154
Epoch 5: loss = 0.134
Epoch 6: loss = 0.118
Epoch 7: loss = 0.108
Epoch 8: loss = 0.100
Epoch 9: loss = 0.091
Epoch 10: loss = 0.085
Train Finished!
Accuracy: 0.960


# TODO

- [ ] Dropout
